# **1. Simulation de portefeuilles statistiquement équivalents**



L’idée : à partir d’une estimation moyenne/variance, on génère plusieurs jeux de données simulées (statistiquement équivalents) par tirage bootstrap, puis on calcule les portefeuilles optimaux correspondants.

## 1.1 Librairies<a id="2.1"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp

## 1.2 Fonction pour calculer portefeuille optimal

Minimiser la variance pour un rendement cible


L’optimisation de portefeuille moderne (Markowitz, 1952) vise à déterminer la composition d’un portefeuille qui **minimise le risque** (mesuré par la variance ou l’écart-type des rendements) pour un **niveau donné de rendement attendu**.

On considère \( n \) actifs, avec :

$$
\boldsymbol{\mu} \in \mathbb{R}^n \quad \text{: vecteur des rendements espérés}
$$

$$
\Sigma \in \mathbb{R}^{n \times n} \quad \text{: matrice de covariance des rendements}
$$

$$
\boldsymbol{w} \in \mathbb{R}^n \quad \text{: vecteur des poids du portefeuille}
$$

avec la contrainte de somme des poids :

$$
\sum_{i=1}^n w_i = 1
$$

---



On cherche à minimiser le risque :

$$
\min_{\boldsymbol{w}} \quad \boldsymbol{w}^\top \Sigma \boldsymbol{w}
$$

Sous les contraintes suivantes :

$$
\boldsymbol{w}^\top \boldsymbol{1} = 1
$$

$$
\boldsymbol{w}^\top \boldsymbol{\mu} = \mu_p
$$

$$
\boldsymbol{w} \geq \boldsymbol{0}
$$

où :

𝜇𝑝 est le rendement cible du portefeuille,


1 est un vecteur colonne de 1 de dimension 
𝑛
,


w≥0 impose qu’aucune vente à découvert n’est autorisée (contraintes de positivité des poids).

La frontière efficiente correspond à l’ensemble des portefeuilles optimaux pour chaque valeur possible de 
𝜇
𝑝

​
 . Elle représente le compromis optimal entre rendement espéré et risque.

In [ ]:
def optimal_portfolio(mu, Sigma, target_return):
    n = len(mu)
    w = cp.Variable(n)
    risk = cp.quad_form(w, Sigma)
    ret = mu @ w
    constraints = [cp.sum(w) == 1, ret == target_return, w >= 0]
    prob = cp.Problem(cp.Minimize(risk), constraints)
    prob.solve()
    return w.value

## 1.3 Data

In [ ]:
# Simuler un jeu de données financier
np.random.seed(42)
n_assets = 5
n_obs = 250

# Simulation données avec moyenne & covariance "réelles"
mu_true = np.array([0.12, 0.10, 0.15, 0.09, 0.11])
Sigma_true = np.array([
    [0.10, 0.02, 0.04, 0.01, 0.03],
    [0.02, 0.08, 0.02, 0.01, 0.02],
    [0.04, 0.02, 0.12, 0.03, 0.04],
    [0.01, 0.01, 0.03, 0.07, 0.02],
    [0.03, 0.02, 0.04, 0.02, 0.09]
])

returns = np.random.multivariate_normal(mu_true, Sigma_true, size=n_obs)

# Estimation à partir des données simulées
mu_hat = returns.mean(axis=0)
Sigma_hat = np.cov(returns, rowvar=False)


## 1.4 Portefeuille optimal et Bootstrap

In [ ]:


# Portefeuille optimal sur estimation initiale
target_return = 0.11
w_opt = optimal_portfolio(mu_hat, Sigma_hat, target_return)

print("Portefeuille optimal estimé :", w_opt)

# Bootstrap pour générer portefeuilles équivalents
n_bootstrap = 100
weights_bootstrap = []

for _ in range(n_bootstrap):
    sample_indices = np.random.choice(n_obs, n_obs, replace=True)
    sample_returns = returns[sample_indices]
    mu_b = sample_returns.mean(axis=0)
    Sigma_b = np.cov(sample_returns, rowvar=False)
    try:
        w_b = optimal_portfolio(mu_b, Sigma_b, target_return)
        weights_bootstrap.append(w_b)
    except:
        pass

weights_bootstrap = np.array(weights_bootstrap)

# Visualisation des poids bootstrap
plt.boxplot(weights_bootstrap, labels=[f"Actif {i+1}" for i in range(n_assets)])
plt.title("Distribution des poids des portefeuilles optimaux bootstrap")
plt.show()
#2. Estimation de la frontière efficiente resamplée
#L’idée : on calcule la moyenne des portefeuilles optimaux obtenus sur plusieurs échantillons bootstrap — cela stabilise la frontière et limite l’impact des erreurs d’estimation.


# Calcul des portefeuilles bootstrap (déjà fait ci-dessus dans weights_bootstrap)

# Moyenne des poids bootstrap
w_resampled = np.mean(weights_bootstrap, axis=0)

print("Portefeuille resamplé moyen :", w_resampled)

# Rendement et risque du portefeuille resamplé
ret_resampled = mu_hat @ w_resampled
risk_resampled = np.sqrt(w_resampled @ Sigma_hat @ w_resampled)

print(f"Rendement resamplé : {ret_resampled:.4f}, Risque resamplé : {risk_resampled:.4f}")


# **2. Estimation de la frontière efficiente resamplée**




L’idée : on calcule la moyenne des portefeuilles optimaux obtenus sur plusieurs échantillons bootstrap — cela stabilise la frontière et limite l’impact des erreurs d’estimation.



In [ ]:
# Calcul des portefeuilles bootstrap (déjà fait ci-dessus dans weights_bootstrap)

# Moyenne des poids bootstrap
w_resampled = np.mean(weights_bootstrap, axis=0)

print("Portefeuille resamplé moyen :", w_resampled)

# Rendement et risque du portefeuille resamplé
ret_resampled = mu_hat @ w_resampled
risk_resampled = np.sqrt(w_resampled @ Sigma_hat @ w_resampled)

print(f"Rendement resamplé : {ret_resampled:.4f}, Risque resamplé : {risk_resampled:.4f}")

# **3. Estimation bootstrap de la frontière efficiente complète**




Ici, on génère plusieurs points de rendement cible et on applique la procédure bootstrap sur chacun pour estimer une frontière efficiente "resamplée".


In [ ]:
target_returns = np.linspace(0.08, 0.16, 10)

frontiers = []
frontiers_resampled = []

for r in target_returns:
    w_list = []
    for _ in range(n_bootstrap):
        sample_indices = np.random.choice(n_obs, n_obs, replace=True)
        sample_returns = returns[sample_indices]
        mu_b = sample_returns.mean(axis=0)
        Sigma_b = np.cov(sample_returns, rowvar=False)
        try:
            w_b = optimal_portfolio(mu_b, Sigma_b, r)
            w_list.append(w_b)
        except:
            pass
    liste_filtrée = [x for x in w_list if x is not None]
    arr = np.array(liste_filtrée) 

    # frontier bootstrap
    risks = [np.sqrt(w @ Sigma_hat @ w) for w in arr]
    frontiers.append((r, min(risks)))
    w_res = np.mean(arr, axis=0)
    risk_res = np.sqrt(w_res @ Sigma_hat @ w_res)
    frontiers_resampled.append((r, risk_res))
   
frontiers = np.array(frontiers)
frontiers_resampled = np.array(frontiers_resampled)
plt.plot(frontiers[:,1], frontiers[:,0], label="Frontière bootstrap (risque minimal)")
plt.plot(frontiers_resampled[:,1], frontiers_resampled[:,0], label="Frontière resamplée (moyenne poids)")
plt.xlabel("Risque (écart-type)")
plt.ylabel("Rendement espéré")
plt.title("Frontières efficientes bootstrap vs resamplée")
plt.legend()
plt.show()
